# J-Quants API V2 Quick Start Guide

In [ ]:
#@title Initial Setting and Import packages
#@markdown Press the ▶ button on the left, and run the code in Google Colab.

#@markdown First, run this code to import the packages required to use the J-Quants API.

import json
import sys
import requests

from IPython.display import display
import pandas as pd

pd.set_option("display.max_columns", None)

API_URL = "https://api.jquants.com/v2"

## Step1：Procedure to start using J-Quants API (*only for the first time)

**To use J-Quants API, you will need to do the following two things in advance**
 1. Register [J-Quants web site](https://jpx-jquants.com/)
 2. Select a plan（Free, Light, Standard, Premium）

If you have not yet registered or selected a plan, please complete the above two items first.  
Click [here](https://jpx-jquants.com/en/spec/quickstart) for more specific instructions.

## Step2：Get API Key

V2 uses API key authentication.

You can get your API key from the Dashboard page on the J-Quants website.

1. Log in to [J-Quants website](https://jpx-jquants.com/)
2. Access the Dashboard page
3. Get your API key (API keys do not expire)

In [ ]:
#@title **Set API Key**
#@markdown Get your API key from the J-Quants Dashboard and paste it below.

api_key = ""#@param {type: "string"}

if api_key:
    headers = {"x-api-key": api_key}
    print("API key has been set. You are ready to use J-Quants API.")
else:
    print("Please enter your API key.")

## Step3： Let's enjoy J-Quants API.

### Free plan
- Listed Issue Information （/equities/master）
- Stock Prices(OHLC)* （/equities/bars/daily）
- Financial Data （/fins/summary)
- Earnings Calendar （/equities/earnings-calendar）
- Trading Calendar（/markets/calendar）

\*  Historical data of morning and afternoon session's prices is available to only premium plan user.


In [ ]:
#@title Listed Issue Information（/equities/master）

#@markdown - listed issue Information as of the past, the current day, and the next business day can be retrieved.
#@markdown - When acquiring data, issue code (code) or date (date) **can** be specified.

#@markdown （Update timing）
#@markdown - Around 17:30 each business day, and around 8:00 the following business day


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date

res = requests.get(f"{API_URL}/equities/master", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/master", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Stock Prices(OHLC)（/equities/bars/daily）

#@markdown - Stock price consists before and after adjustment of stock splits and reverse stock splits (Rounded to first decimal places)
#@markdown - In your request message, either "code" or "date" **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 17:00


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/equities/bars/daily", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/bars/daily", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Financial Data（/fins/summary）

#@markdown - Obtain quarterly earnings summaries and disclosure information (mainly numerical data) on revisions to earnings and dividend information for listed companies.
#@markdown - In your request message, either "code" or "date" **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 18:00(preliminary) and around 24:30(final)


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date

res = requests.get(f"{API_URL}/fins/summary", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/fins/summary", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Earnings Calendar（/equities/earnings-calendar）

#@markdown （Update timing）
#@markdown - Irregularly around 19:00

#@markdown - It will be updated only when there is an update for companies which end their fiscal year in March or September at [this site](https://www.jpx.co.jp/listing/event-schedules/financial-announcement/index.html).

params = {}

res = requests.get(f"{API_URL}/equities/earnings-calendar", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/earnings-calendar", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Trading Calendar（/markets/calendar）

#@markdown - You can obtain information on market business days, holidays in TSE/OSE market, and the availability of holiday trading in OSE market.
#@markdown - In your request message, either "holiday division" or "from/to" can be specified.

#@markdown （Update timing）
#@markdown - Irregular（The business days and holiday trading dates (tentative) for the following year will be updated around February of each year.）


hol_div = ""#@param ["0", "1", "2", "3"]{allow-input:true}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if hol_div != "":
  params["hol_div"] = hol_div
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/markets/calendar", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/calendar", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

### Light plan
- Trading by Type of Investors（/equities/investor-types）
- TOPIX Prices(OHLC)（/indices/bars/daily/topix）

In [ ]:
#@title Trading by Type of Investors（/equities/investor-types）

#@markdown - Trading by type of investors (stock trading value) can be obtained.
#@markdown - In your request message, either "section" or "from/to" can be specified.

#@markdown （Update timing）
#@markdown - Weekly(4th business day)、around 18:00


section = ""#@param ["TSE1st", "TSE2nd", "TSEMothers", "TSEJASDAQ", "TSEPrime", "TSEStandard", "TSEGrowth", "TokyoNagoya"] {allow-input: true}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if section != "":
  params["section"] = section
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/equities/investor-types", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/investor-types", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())


In [ ]:
#@title TOPIX Prices(OHLC)（/indices/bars/daily/topix）

#@markdown - Available index is TOPIX（Tokyo Stock Price Index）.
#@markdown - In your request message, "from/to" can be specified.

#@markdown （Update timing）
#@markdown - Daily, around 16:30


date_from = ""#@param {type:"string"}
date_to = ""#@param {type:"string"}

params = {}

if date_from != "":
  params["from"] = date_from
if date_to != "":
  params["to"] = date_to

res = requests.get(f"{API_URL}/indices/bars/daily/topix", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/indices/bars/daily/topix", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())


### Standard plan
- Indices（/indices/bars/daily）
- Index Option Prices(OHLC)（/derivatives/bars/daily/options/225）
- Margin Trading Outstandings（/markets/margin-interest）
- Short Sale value and Ratio by Sector（/markets/short-ratio）
- Outstanding Short Selling Positions Reported (/markets/short-sale-report)
- Margin Trading Outstanding (Issues Subject to Daily Publication)(/markets/margin-alert)

In [ ]:
#@title Indices（/indices/bars/daily）

#@markdown - Available various indices OHLC.
#@markdown - In your request message, either "code" or "date" **must** be specified.

#@markdown （Update Timing）
#@markdown - Daily, around 16:30


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/indices/bars/daily", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/indices/bars/daily", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Index Option Prices(OHLC)（/derivatives/bars/daily/options/225）

#@markdown - Information on the OHLC, settlement price, and theoretical price of Nikkei 225 Options can be obtained through this API.
#@markdown - When acquiring data, "date" **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 27:00


date = "" #@param {type:"string"}
params = {}
if date != "":
  params["date"] = date

res = requests.get(f"{API_URL}/derivatives/bars/daily/options/225", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/derivatives/bars/daily/options/225", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Margin Trading Outstandings（/markets/margin-interest）

#@markdown - Weekly margin trading outstandings as of the last business day of each week is available.
#@markdown - This data is also available via [this page](https://www.jpx.co.jp/english/markets/statistics-equities/margin/index.html).
#@markdown - In your request message, either "code" or "date" **must** be specified.

#@markdown （Update timing）
#@markdown - Weekly(2nd business day) around 16:30

code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/markets/margin-interest", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/margin-interest", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Short Sale Value and Ratio by Sector（/markets/short-ratio）

#@markdown - You can obtain daily trading values related to short sale ratios by industry (sector).
#@markdown - This data is also available via [this page](https://www.jpx.co.jp/english/markets/statistics-equities/short-selling/index.html).
#@markdown - When acquiring data, "date" or "s33" **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 16:30

#@markdown （Note）
#@markdown - Industry code 9999 is for stocks that are not included in the 33 industry categories, such as ETFs and REITs.


s33 = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if s33 != "":
  params["s33"] = s33
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/markets/short-ratio", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/short-ratio", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Outstanding Short Selling Positions Reported (/markets/short-sale-report)

#@markdown - This data covers the outstanding short selling position ratio is 0.5% or more of those reported by trading participants in accordance with the "Cabinet Office Order on Restrictions on Securities Transactions".
#@markdown - In your request message, either issue code (code) or disclosed date (disc_date) or calculated date (calc_date) **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 17:30

code = ""#@param {type:"string"}
disc_date = ""#@param {type:"string"}
disc_date_from = ""#@param {type:"string"}
disc_date_to = ""#@param {type:"string"}
calc_date = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if disc_date != "":
  params["disc_date"] = disc_date
if disc_date_from != "":
  params["disc_date_from"] = disc_date_from
if disc_date_to != "":
  params["disc_date_to"] = disc_date_to
if calc_date != "":
  params["calc_date"] = calc_date

res = requests.get(f"{API_URL}/markets/short-sale-report", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/short-sale-report", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Margin Trading Outstanding (Issues Subject to Daily Publication)（/markets/margin-alert）

#@markdown - Only stocks for which the Tokyo Stock Exchange or Japan Securities Finance Co., Ltd (JSF) decide it necessary to disclose Daily margin trading outstandings are included.
#@markdown - In your request message, either "code" or "date" must be specified.

#@markdown （Update timing）
#@markdown - Daily, around 16:30

code = ""#@param {type:"string"}
_date = ""#@param {type:"string"}
_from = ""#@param {type:"string"}
_to = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if _date != "":
  params["date"] = _date
if _from != "":
  params["from"] = _from
if _to != "":
  params["to"] = _to

res = requests.get(f"{API_URL}/markets/margin-alert", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/margin-alert", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

### Premium plan
- Breakdown Trading Data（/markets/breakdown）
- Morning Session Stock Prices(OHLC)（/equities/bars/daily/am）
- Cash Dividend Data（/fins/dividend）
- Financial Statement Data(BS/PL/CF)（/fins/details）
- Futures(OHLC)（/derivatives/bars/daily/futures）
- Options(OHLC)（/derivatives/bars/daily/options）

In [ ]:
#@title Breakdown Trading Data（/markets/breakdown）

#@markdown - Detail Breakdown Trading Data is extracted from daily trading values and volumes (only regular trading sessions on the TSE market) per TSE-listed issue based on flags for margin transactions and short selling that are attached to orders at the time of placement.
#@markdown - In your request message, either "code" or "date" **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 18:00


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/markets/breakdown", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/breakdown", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Morning Session Stock Prices(OHLC)（/equities/bars/daily/am）

#@markdown - You can obtain the morning session's high, low, opening, and closing prices for individual stocks as quick updates at noon.
#@markdown - In your request message, either "code" **can** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 12:00


code = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code

res = requests.get(f"{API_URL}/equities/bars/daily/am", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/bars/daily/am", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Cash Dividend Data（/fins/dividend）

#@markdown - Provides information on dividends (determined and forecast) per share of listed companies, record date, ex-rights date, and payable date.
#@markdown - When acquiring data, issue code (code) or date (date) **can** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 18:00


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/fins/dividend", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/fins/dividend", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Financial Statement Data(BS/PL/CF) (/fins/details)

#@markdown - You can obtain the entries in the balance sheet and income statement of a listed companies in their quarterly financial information.
#@markdown - In this code, you can switch how the output results are displayed by specifying the format_type: flat: all items are displayed in the header; non-flat: detailed items are consolidated in one column.
#@markdown - In your request message, either issue code (code) or disclosure date (date) **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 18:00(preliminary) and around 24:30(final)


format_type = "non-flat"#@param ["non-flat", "flat"]
code = ""#@param {type:"string"}
date = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date

res = requests.get(f"{API_URL}/fins/details", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/fins/details", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  if format_type == "non-flat":
    df = pd.DataFrame(data)
  else:
    df = pd.json_normalize(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Futures(OHLC)(/derivatives/bars/daily/futures)

#@markdown - Information on the OHLC, settlement price, and theoretical price of futures can be obtained through this API. 
#@markdown - In your request message, date (date) **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 27:00

category = ""#@param {type:"string"}
date = ""#@param {type:"string"}
contract_flag = ""#@param{type:"string"}

params = {}
if category != "":
  params["category"] = category
if date != "":
  params["date"] = date
if contract_flag != "":
  params["contract_flag"] = contract_flag

res = requests.get(f"{API_URL}/derivatives/bars/daily/futures", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/derivatives/bars/daily/futures", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Options(OHLC)(/derivatives/bars/daily/options)

#@markdown - Information on the OHLC, settlement price, and theoretical price of options can be obtained through this API.
#@markdown - In your request message, date (date) **must** be specified.

#@markdown （Update timing）
#@markdown - Daily, around 27:00

category = ""#@param {type:"string"}
date = ""#@param {type:"string"}
contract_flag = ""#@param{type:"string"}

params = {}
if category != "":
  params["category"] = category
if date != "":
  params["date"] = date
if contract_flag != "":
  params["contract_flag"] = contract_flag

res = requests.get(f"{API_URL}/derivatives/bars/daily/options", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/derivatives/bars/daily/options", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

### Light plan or higher + Add-on contract required
- Stock Prices (Minute Bars)（/equities/bars/minute）

\*  Stock price minute/tick add-on contract is required. Data available for the past 2 years.

In [ ]:
#@title Stock Prices (Minute Bars)（/equities/bars/minute）

#@markdown - Endpoint to retrieve minute-level stock price data. You can get OHLC (Open, High, Low, Close) data in 1-minute intervals, along with volume and trading value.
#@markdown - In your request message, either "code" or "date" **must** be specified.
#@markdown - Data available for the past 2 years.

#@markdown （Update timing）
#@markdown - Daily, around 16:30

#@markdown （Additional Notes）
#@markdown - Stock price minute/tick add-on contract is required.
#@markdown - For bulk data retrieval of all stocks, using the Bulk API is recommended.

code = ""#@param {type:"string"}
date = "20250122"#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/equities/bars/minute", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/bars/minute", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

### Bulk API (CSV Download) for Light plan or higher
- List of Downloadable Files（/bulk/list）
- Get File Download URL（/bulk/get）

Using the Bulk API, you can efficiently download large amounts of data in CSV format.

\*  CSV download is not available for Free plan users.

In [ ]:
#@title List of Downloadable Files（/bulk/list）

#@markdown - This endpoint retrieves a list of files available for download in CSV format.
#@markdown - Specify the dataset you want to retrieve using the endpoint parameter.

#@markdown （Specifiable endpoint examples）
#@markdown - /equities/bars/daily (Stock OHLC)
#@markdown - /equities/bars/minute (Stock Minute Bars)
#@markdown - /fins/summary (Financial Data)
#@markdown - /indices/bars/daily/topix (TOPIX OHLC)
#@markdown - etc.

#@markdown For a complete list of specifiable endpoints, please see [here](https://jpx-jquants.com/en/spec/bulk-list/endpoints).

endpoint = "/equities/bars/daily"#@param {type:"string"}

params = {}
if endpoint != "":
  params["endpoint"] = endpoint

res = requests.get(f"{API_URL}/bulk/list", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title Get File Download URL（/bulk/get）

#@markdown - Specify the Key obtained from /bulk/list to get a signed URL for downloading the CSV file.
#@markdown - The obtained URL is valid for approximately 5 minutes. Please complete the download within this time.

#@markdown （How to use）
#@markdown 1. First, retrieve the list of downloadable files using /bulk/list
#@markdown 2. Check the Key of the file you want to download from the retrieved list
#@markdown 3. Enter that Key below and execute
#@markdown 4. Access the obtained URL to download the CSV

key = ""#@param {type:"string"}

params = {}
if key != "":
  params["key"] = key

res = requests.get(f"{API_URL}/bulk/get", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  download_url = d.get("url", "")
  print("Download URL (Valid for approximately 5 minutes):")
  print(download_url)
  print("\n* Please access the URL above to download the CSV file.")
else:
  print(res.json())